In [ ]:
import os
import csv
import requests
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import datetime as dt
from finta import TA
import hvplot.pandas
from pathlib import Path
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder

#Subsequent imports
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import LabelEncoder

from collections import deque

class PSAR:

  def __init__(self, init_af=0.02, max_af=0.2, af_step=0.02):
    self.max_af = max_af
    self.init_af = init_af
    self.af = init_af
    self.af_step = af_step
    self.extreme_point = None
    self.high_price_trend = []
    self.low_price_trend = []
    self.high_price_window = deque(maxlen=2)
    self.low_price_window = deque(maxlen=2)

    # Lists to track results
    self.psar_list = []
    self.af_list = []
    self.ep_list = []
    self.high_list = []
    self.low_list = []
    self.trend_list = []
    self._num_days = 0

  def calcPSAR(self, high, low):
    if self._num_days >= 3:
      psar = self._calcPSAR()
    else:
      psar = self._initPSARVals(high, low)

    psar = self._updateCurrentVals(psar, high, low)
    self._num_days += 1

    return psar

  def _initPSARVals(self, high, low):
    if len(self.low_price_window) <= 1:
      self.trend = None
      self.extreme_point = high
      return None

    if self.high_price_window[0] < self.high_price_window[1]:
      self.trend = 1
      psar = min(self.low_price_window)
      self.extreme_point = max(self.high_price_window)
    else: 
      self.trend = 0
      psar = max(self.high_price_window)
      self.extreme_point = min(self.low_price_window)

    return psar

  def _calcPSAR(self):
    prev_psar = self.psar_list[-1]
    if self.trend == 1: # Up
      psar = prev_psar + self.af * (self.extreme_point - prev_psar)
      psar = min(psar, min(self.low_price_window))
    else:
      psar = prev_psar - self.af * (prev_psar - self.extreme_point)
      psar = max(psar, max(self.high_price_window))

    return psar

  def _updateCurrentVals(self, psar, high, low):
    if self.trend == 1:
      self.high_price_trend.append(high)
    elif self.trend == 0:
      self.low_price_trend.append(low)

    psar = self._trendReversal(psar, high, low)

    self.psar_list.append(psar)
    self.af_list.append(self.af)
    self.ep_list.append(self.extreme_point)
    self.high_list.append(high)
    self.low_list.append(low)
    self.high_price_window.append(high)
    self.low_price_window.append(low)
    self.trend_list.append(self.trend)

    return psar

  def _trendReversal(self, psar, high, low):
    # Checks for reversals
    reversal = False
    if self.trend == 1 and psar > low:
      self.trend = 0
      psar = max(self.high_price_trend)
      self.extreme_point = low
      reversal = True
    elif self.trend == 0 and psar < high:
      self.trend = 1
      psar = min(self.low_price_trend)
      self.extreme_point = high
      reversal = True

    if reversal:
      self.af = self.init_af
      self.high_price_trend.clear()
      self.low_price_trend.clear()
    else:
        if high > self.extreme_point and self.trend == 1:
          self.af = min(self.af + self.af_step, self.max_af)
          self.extreme_point = high
        elif low < self.extreme_point and self.trend == 0:
          self.af = min(self.af + self.af_step, self.max_af)
          self.extreme_point = low

    return psar


# Enable the Matplotlib property to allow diagrams to display in Jupyter Notebook
%matplotlib inline

In [ ]:
#dfcrypto = yf.download('BTC-USD LTC-USD XRP-USD DOGE-USD', start='2013-12-14', end='2022-11-27')

lstCryptos=['BTC-USD', 'LTC-USD', 'XRP-USD', 'DOGE-USD']

lstDfs=[]

dfCombined=pd.DataFrame()


In [ ]:
for crypto in lstCryptos:

    dfcrypto = yf.download(crypto, start='2013-12-14', end='2022-11-27')
    dfcrypto['ClosePct']=(dfcrypto['Close']-dfcrypto['Low'])/(dfcrypto['High']-dfcrypto['Low'])
    dfcrypto['Range']=dfcrypto['High']-dfcrypto['Low']
    dfcrypto['RangeMa']=dfcrypto['Range'].rolling(30).mean()
    dfcrypto['RangeSD']=dfcrypto['Range'].rolling(30).std()
    dfcrypto['RangeZscore']=(dfcrypto['Range']-dfcrypto['RangeMa'])/dfcrypto['RangeSD']
    dfcrypto['Ticker']=crypto
    dfcrypto['Y']=dfcrypto['Close'].pct_change().shift(-1)
    display(dfcrypto)
    dfcrypto=dfcrypto.drop(columns=['Open','High','Low','Close','Adj Close','Volume','Range','RangeMa','RangeSD'])
    lstDfs.append(dfcrypto)
    del(dfcrypto)
                           
dfCombined=pd.concat(lstDfs,axis=0)
dfCombined


In [ ]:
dfCombined=dfCombined.reset_index()
dfCombined

In [ ]:
dfCombined=dfCombined.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])

In [ ]:
dfCombined.dropna(inplace=True)
dfCombined

In [ ]:
dfCombined.hvplot(x='RangeZscore', y='Y', kind='kde', figsize=(30,20))

In [ ]:
correlation = dfCombined.corr()
correlation

In [ ]:
sns.heatmap(correlation, vmin=-1, vmax=1)

In [ ]:
short_window = 30
long_window=90

previous_price =0


lstdf=[]
dfCombined2=pd.DataFrame()

for crypto in lstCryptos:
    dfcrypto= yf.download(crypto, start='2013-12-14', end='2022-11-27')
    dfcrypto['Actual Returns']=dfcrypto['Close'].pct_change()
    dfcrypto['SMA_Fast']= dfcrypto['Close'].rolling(window=short_window).mean()
    dfcrypto['SMA_Slow']= dfcrypto['Close'].rolling(window=long_window).mean()
    dfcrypto['Signal']=0.0
    dfcrypto.loc[(dfcrypto['Actual Returns'] >= 0), 'Signal'] = 1
    dfcrypto.loc[(dfcrypto['Actual Returns'] < 0), 'Signal'] = -1
    dfcrypto['Strategy Returns']=dfcrypto['Actual Returns'] * dfcrypto['Signal'].shift()
    dfcrypto['Ticker']=crypto
    dfcrypto=dfcrypto.drop(columns=['Open','High','Low','Adj Close','Volume'])
    dfcrypto['Trade_Type'] = np.nan
    for index, row in dfcrypto.iterrows():
        if previous_price == 0:
            dfcrypto.loc[index, 'Trade_Type'] = 'Buy'
        elif row["Close"] < previous_price:
            dfcrypto.loc[index, "Trade_Type"] = 'Buy'
        else:
            dfcrypto.loc[index, 'Trade_Type'] ='Hold'
        previous_price = row['Close']
        if index == dfcrypto.index[-1]:
            dfcrypto.loc[index, 'Trade_Type'] = 'Sell'
    display(dfcrypto)
    lstdf.append(dfcrypto)
    del(dfcrypto)
    
    
dfCombined2=pd.concat(lstdf, axis=0)
dfCombined2


dfCombined2=dfCombined2.reset_index()
dfCombined2
    
dfCombined2=dfCombined2.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])

dfCombined2.dropna(inplace=True)
#dfCombined2=dfCombined2.reset_index()
#dfCombined2=dfCombined2.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])

dfCombined2



In [ ]:
## Spilit the data into training and testing dataset 
X= dfCombined2[['SMA_Fast','SMA_Slow']].shift().dropna()
# review DataFrame
X


In [ ]:
# Creat the target set selecting the signal column and assigning it to y
y = dfCombined2['Signal']
y.value_counts()

# Select the begining of the training 
training_begin = X.index.min()
print(training_begin)

# Select the ending period for the training data with an offset of 3 months
training_end = (pd.Timestamp("2014-11-05") + pd.DateOffset(months=60))
print(training_end)

# Generate the X_train,y_train, X_test and y_test DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Standardize the Data
scaler= StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Creat the classifier model 
svm_model = svm.SVC()
svm_model = svm_model.fit(X_train_scaled,y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_pred[:10]


In [ ]:
svm_testing_report = classification_report(y_test, svm_pred)
print(svm_testing_report)

In [ ]:
## Backtest the new model to evaluate 
# Creat a prediction DataFrame

dfprediction = pd.DataFrame(index=X_test.index)
dfprediction['Predicted']= svm_pred
dfprediction['Actual Returns']=dfCombined2['Actual Returns']
dfprediction['Strategy Returns']=dfprediction['Actual Returns']*dfprediction['Predicted']
display(dfprediction)


In [ ]:
(1 + dfprediction[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(
    figsize=(40,20)
)

In [ ]:
short_window = 20
long_window=100

previous_price =0


DFlsts=[]
dfCombined3=pd.DataFrame()

for crypto in lstCryptos:
    dfcrypto= yf.download(crypto, start='2013-12-14', end='2022-11-27')
    dfcrypto['Actual Returns']=dfcrypto['Close'].pct_change()
    dfcrypto['SMA_Fast']= dfcrypto['Close'].rolling(window=short_window).mean()
    dfcrypto['SMA_Slow']= dfcrypto['Close'].rolling(window=long_window).mean()
    dfcrypto['SMA Signal']=0.0
    dfcrypto.loc[(dfcrypto['Actual Returns'] >= 0), 'SMA Signal'] = 1
    dfcrypto.loc[(dfcrypto['Actual Returns'] < 0), 'SMA Signal'] = -1
    dfcrypto['Strategy Returns']=dfcrypto['Actual Returns'] * dfcrypto['SMA Signal'].shift()
    dfcrypto['Ticker']=crypto
    dfcrypto=dfcrypto.drop(columns=['Open','High','Low','Adj Close','Volume'])
    dfcrypto['Trade_Type'] = np.nan
    for index, row in dfcrypto.iterrows():
        if previous_price == 0:
            dfcrypto.loc[index, 'Trade_Type'] = 'Buy'
        elif row["Close"] < previous_price:
            dfcrypto.loc[index, "Trade_Type"] = 'Buy'
        else:
            dfcrypto.loc[index, 'Trade_Type'] ='Hold'
        previous_price = row['Close']
        if index == dfcrypto.index[-1]:
            dfcrypto.loc[index, 'Trade_Type'] = 'Sell'
    display(dfcrypto)
    DFlsts.append(dfcrypto)
    del(dfcrypto)
    
    
dfCombined3=pd.concat(DFlsts, axis=0)
dfCombined3


dfCombined3=dfCombined3.reset_index()
dfCombined3
    
dfCombined3=dfCombined3.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])
dfCombined3.dropna(inplace=True)
display(dfCombined3)

## Spilit the data into training and testing dataset 
X=dfCombined3[['SMA_Fast','SMA_Slow']].copy().dropna()
# review DataFrame
X




In [ ]:
# Creat the target set selecting the signal column and assigning it to y
y = dfCombined3['SMA Signal']
y.value_counts()

# Select the begining of the training 
training_begin = X.index.min()
print(training_begin)

# Select the ending period for the training data with an offset of 3 months
training_end = (pd.Timestamp("2014-11-05") + pd.DateOffset(months=36))
print(training_end)

# Generate the X_train,y_train, X_test and y_test DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Standardize the Data
scaler= StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
y.shape

In [ ]:
# Initiate the model logistic_Regression_model instance
logistic_Regression_model= LogisticRegression()
# Fit the model using the training data
logistic_Regression_model =  logistic_Regression_model.fit(X_train_scaled,y_train)
# Use the testing dataset to generate the predictions for the new model
log_pred = logistic_Regression_model.predict(X_test_scaled)

# Review the model's predicted values
log_pred[:10]


In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
training_report = classification_report(y_test, log_pred) 

# Print the classification report
print(training_report)


In [ ]:
# Create a predictions DataFrame
prediction_df= pd.DataFrame(index=X_test.index)
# Add the  logistic_Regression_model predictions to the DataFrame
prediction_df['prediction']= log_pred

# Add the actual returns to the DataFrame
prediction_df['Actual Returns']= dfCombined3['Actual Returns']

# Add the strategy returns to the DataFrame
prediction_df['Strategy Returns']=(prediction_df['Actual Returns']*prediction_df['prediction'])

# Review the DataFrame
prediction_df


In [ ]:
# plot predictions_df
(1+prediction_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(figsize=(30,20))


In [ ]:
dfCombined3.reset_index(inplace=True)

In [ ]:
dfCombined3


In [ ]:
X.shape

In [ ]:
## Generate Exponential Moving day Average of 20 and 100 days

# Create 20 days exponential moving average column
dfCombined3['EMA20'] = dfCombined3['Close'].ewm(span = 20, adjust = False).mean()

# Create 100 days exponential moving average column
dfCombined3['EMA100'] = dfCombined3['Close'].ewm(span = 100, adjust = False).mean()

# create a new column 'EMA Signal' such that if 20-day EMA is greater   # than 100-day EMA then set Signal as 1 else 0
dfCombined3['EMA Signal'] = 0.0  
dfCombined3['EMA Signal'] = np.where(dfCombined3['EMA20'] > dfCombined3['EMA100'], 1.0, 0.0)

# create a new column 'Entry/Exit' which is a day-to-day difference of # the 'Signal' column
dfCombined3['Entry/Exit']= dfCombined3['EMA Signal'].diff()



In [ ]:
# Visualize entry position relative to close price
entry = dfCombined3[dfCombined3["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = dfCombined3[ dfCombined3["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = dfCombined3[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs =  dfCombined3[["EMA20", "EMA100"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot


In [ ]:
dfCombined3=dfCombined3.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])
dfCombined3= dfCombined3[['Close','Actual Returns','SMA_Fast','SMA_Slow','SMA Signal','Strategy Returns','EMA20','EMA100','EMA Signal','Entry/Exit','Trade_Type']]
dfCombined3

In [ ]:
## Generate Parabolic Sar to indicate market down and up trends

In [ ]:
indic=PSAR()
ticker = 'BTC-USD'
yfObj = yf.Ticker(ticker)
data1 =yf.download(ticker, start='2013-12-14', end='2022-11-26') 
#yfObj.history(start='2013-12-14', end='2022-11-27')
data1['Ticker']=ticker
display(data1)

data1['PSAR'] = data1.apply(
    lambda x: indic.calcPSAR(x['High'],x['Low']), axis= 1)
data1['EP']=indic.ep_list
data1['Trend']=indic.trend_list
data1['AF'] = indic.af_list
data1=data1.drop(columns=['Open','High','Low','Adj Close','Volume'])
display(data1)



psar_bull = data1.loc[data1['Trend']==1]['PSAR']
psar_bear = data1.loc[data1['Trend']==0]['PSAR']

BTC_close= data1[['Close']].hvplot(
    line_color='lightgrey',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'BTC Market Trend', 
    fontsize = 20,
    width=1600,
    height=800
)

BTC_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

BTC_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

BTC_PSAR_UP = BTC_close * BTC_Up_Trend * BTC_Down_Trend
BTC_PSAR_UP






In [ ]:
buy_signals = data1.loc[data1['Trend'].diff()==1]['Close']
short_signals = data1.loc[data1['Trend'].diff()==-1]['Close']

BTC_close= data1[['Close']].hvplot(
    line_color='black',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'BTC Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

BTC_buy= buy_signals.hvplot.scatter(
    line_color='blue',
    label='Buy',
    marker='^',
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'XRP Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

BTC_short=short_signals.hvplot.scatter(
    line_color='yellow',
    label='Short',
    marker='v', 
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'XRP Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

BTC_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=600,
    height=800
)

BTC_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

BTC_buy_short_plot = BTC_close * BTC_buy * BTC_short * BTC_Up_Trend *BTC_Down_Trend
BTC_buy_short_plot

In [ ]:
indic=PSAR()

ticker = 'LTC-USD'
yfObj = yf.Ticker(ticker)
data2 = yf.download(ticker, start='2013-12-14', end='2022-11-27') 
data2['Ticker']=ticker
display(data2)

data2['PSAR'] = data2.apply(
    lambda x: indic.calcPSAR(x['High'],x['Low']), axis= 1)
data2['EP']=indic.ep_list
data2['Trend']=indic.trend_list
data2['AF'] = indic.af_list
data2=data2.drop(columns=['Open','High','Low','Adj Close','Volume'])
display(data2)

psar_bull = data2.loc[data2['Trend']==1]['PSAR']
psar_bear = data2.loc[data2['Trend']==0]['PSAR']

LTC_close= data2[['Close']].hvplot(
    line_color='lightgrey',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'LTC Market Trend', 
    fontsize = 20,
    width=1600,
    height=800
)

LTC_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

LTC_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

LTC_PSAR_UP = LTC_close * LTC_Up_Trend * LTC_Down_Trend
LTC_PSAR_UP


In [ ]:
buy_signals = data2.loc[data2['Trend'].diff()==1]['Close']
short_signals = data2.loc[data2['Trend'].diff()==-1]['Close']

LTC_close= data2[['Close']].hvplot(
    line_color='black',
    line_width=1,
    ylabel = 'Price in USD',
    title= ' LTC Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

LTC_buy= buy_signals.hvplot.scatter(
    line_color='blue',
    label='Buy',
    marker='^',
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'LTC Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

LTC_short=short_signals.hvplot.scatter(
    line_color='yellow',
    label='Short',
    marker='v', 
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'LTC Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

LTC_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

LTC_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

LTC_buy_short_plot = LTC_close * LTC_buy * LTC_short * LTC_Up_Trend * LTC_Down_Trend
LTC_buy_short_plot

In [ ]:
indic=PSAR()

ticker = 'XRP-USD'
yfObj = yf.Ticker(ticker)
data3 = yf.download(ticker, start='2017-11-09', end='2022-11-27') 
data3['Ticker']=ticker
display(data3)

data3['PSAR'] = data3.apply(
    lambda x: indic.calcPSAR(x['High'],x['Low']), axis= 1)
data3['EP']=indic.ep_list
data3['Trend']=indic.trend_list
data3['AF'] = indic.af_list
data3=data3.drop(columns=['Open','High','Low','Adj Close','Volume'])
display(data3)

psar_bull = data3.loc[data3['Trend']==1]['PSAR']
psar_bear = data3.loc[data3['Trend']==0]['PSAR']

XRP_close= data3[['Close']].hvplot(
    line_color='lightgrey',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'XRP Market Trend', 
    fontsize = 20,
    width=1600,
    height=800
)
XRP_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

XRP_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Dpwm Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 


XRP_PSAR_UP = XRP_close * XRP_Up_Trend * XRP_Down_Trend

XRP_PSAR_UP


In [ ]:
buy_signals = data3.loc[data3['Trend'].diff()==1]['Close']
short_signals = data3.loc[data3['Trend'].diff()==-1]['Close'] 

XRP_close= data3[['Close']].hvplot(
    line_color='black',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'XRP Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

XRP_buy= buy_signals.hvplot.scatter(
    line_color='blue',
    label='Buy',
    marker='^',
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'XRP Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

XRP_short=short_signals.hvplot.scatter(
    line_color='yellow',
    label='Short',
    marker='v', 
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'XRP Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

XRP_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

XRP_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

XRP_buy_short_plot = XRP_close * XRP_buy * XRP_short * XRP_Up_Trend * XRP_Down_Trend
XRP_buy_short_plot


In [ ]:
indic=PSAR()

ticker = 'DOGE-USD'
yfObj = yf.Ticker(ticker)
data4 = yf.download(ticker, start='2017-11-09', end='2022-11-27') 
data4['Ticker']=ticker
display(data4)

data4['PSAR'] = data4.apply(
    lambda x: indic.calcPSAR(x['High'],x['Low']), axis= 1)
data4['EP']=indic.ep_list
data4['Trend']=indic.trend_list
data4['AF'] = indic.af_list
data4=data4.drop(columns=['Open','High','Low','Adj Close','Volume'])
display(data4)

#colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

psar_bull = data4.loc[data4['Trend']==1]['PSAR']
psar_bear = data4.loc[data4['Trend']==0]['PSAR']


DOGE_close= data4[['Close']].hvplot(
    line_color='lightgrey',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'DogeCoin Market Trend', 
    fontsize = 20,
    width=1600,
    height=800
)

DOGE_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

DOGE_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

doge_PSAR_UP = DOGE_close * DOGE_Up_Trend * DOGE_Down_Trend
doge_PSAR_UP


    

In [ ]:
buy_signals = data4.loc[data4['Trend'].diff()==1]['Close']
short_signals = data4.loc[data4['Trend'].diff()==-1]['Close']


            
DOGE_close= data4[['Close']].hvplot(
    line_color='black',
    line_width=1,
    ylabel = 'Price in USD',
    title= 'DogeCoin Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

DOGE_buy= buy_signals.hvplot.scatter(
    line_color='blue',
    label='Buy',
    marker='^',
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'DogeCoin Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

DOGE_short=short_signals.hvplot.scatter(
    line_color='yellow',
    label='Short',
    marker='v',
    size=400,
    legend='top_right',
    ylabel = 'Price in USD',
    title= 'DogeCoin Buy and Short Signal', 
    fontsize = 20,
    width=1600,
    height=800
)

DOGE_Up_Trend=psar_bull.hvplot.scatter(
    line_color='red',
    line_width=1,
    label='Up Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
)

DOGE_Down_Trend=psar_bear.hvplot.scatter(
    line_color='orange',
    line_width=1,
    label='Down Trend',
    legend='top_right',
    ylabel= 'Price in USD',
    fontsize = 20,
    width=1600,
    height=800
) 

doge_buy_short_plot = DOGE_close * DOGE_buy * DOGE_short * DOGE_Up_Trend * DOGE_Down_Trend
doge_buy_short_plot


In [ ]:
df=pd.concat([data1,data2,data3,data4], axis=0)
df.reset_index(inplace=True)
df=df.sort_values(['Date','Ticker']).set_index(['Date','Ticker'])
df.dropna(inplace=True)
df
dfCombined3=dfCombined3.append(df)
dfCombined3

In [ ]:
dfCombined3.reset_index(inplace=True)


In [ ]:
dfCombined2['Trade_Type'].value_counts()

In [ ]:
dfCombined3['Trade_Type'].value_counts()

In [ ]:
dfCombined3['SMA Signal'].value_counts()

In [ ]:
dfCombined3['EMA Signal'].value_counts()

In [ ]:
# Define y output variable
y1 = dfCombined3['Trade_Type']
y1.value_counts()


In [ ]:
y1.shape

In [ ]:
# Convert trading type to integers
encoder = LabelEncoder()
encoder.fit(y1)
encoder_y1 = encoder.transform(y1)
encoder_y1


In [ ]:
encoder_y1.shape

In [ ]:
number_of_classes = len(list(y1.drop_duplicates()))
number_of_classes

In [ ]:
# Convert labeled integers to a Keras `categorical` data type
y1_categorical = to_categorical(encoder_y1, num_classes=number_of_classes)
y1_categorical.shape


In [ ]:
# Specify X (predictor) variables
X1 = dfCombined3[['Close']].copy().dropna()
X1


In [ ]:
X1.shape

In [ ]:
# Split into training and testing windows
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1_categorical, random_state=1)

In [ ]:
# Create a StandardScaler instance
X1_scaler = StandardScaler()

# Fit the scaler to the features training dataset

X1_scaler.fit(X1_train)

# Scale both the training and testing data from the features dataset
X1_train_scaled = X1_scaler.transform(X1_train)
X1_test_scaled = X1_scaler.transform(X1_test)


In [ ]:
pd.DataFrame(y_train).sum()

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X1_train.iloc[0])
number_input_features

In [ ]:

model = Sequential()
model.add(Dense(40, input_dim=number_input_features, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))


In [ ]:
# Add the final output layer
model.add(Dense(4, activation='sigmoid'))

In [ ]:
# Compile the model (with multi-class specific parameters)
model.compile(loss='categorical_crossentropy', 
              optimizer= 'adam', 
              metrics=['categorical_accuracy'])


In [ ]:
# Summarise the structure of the model
model.summary()


In [ ]:
# Fit the modelmodel
number_of_epochs=100
model.fit(X1_train,y1_train, 
                    epochs=number_of_epochs,
                    batch_size=1000,
                    validation_split=0.2,
                    shuffle=True)

In [ ]:
model_history = model.history.history


In [ ]:
model_history.keys()

In [ ]:
model_history['categorical_accuracy']

In [ ]:
# Evaluate model on the test data
model.evaluate(X1_test,y1_test, verbose=2)


In [ ]:
# Save predictions on the test data
predictions = model.predict(X1_test)
predictions


In [ ]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
# Convert most likely category back to original labels
most_likely = encoder.inverse_transform((most_likely))
most_likely


In [ ]:
# Evaluate prediction balan
pd.DataFrame(most_likely).value_counts()


In [ ]:
training_results = pd.DataFrame(index=range(1, number_of_epochs+1))
training_results['Training'] = model_history['categorical_accuracy']
training_results['Validation'] = model_history['val_categorical_accuracy']
training_results.hvplot(title = 'Performance Across Training and Validation', figsize=(30,20))